In [1]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.45.3
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: 
Author-email: Tim Dettmers <dettmers@cs.washington.edu>
License: MIT License
        
        Copyright (c) Facebook, Inc. and its affiliates.
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED 

In [11]:
import requests
import json
import re

# Define the FastAPI endpoint URL
FASTAPI_URL = "http://127.0.0.1:8000/generate"

# Define the prompt
prompt = """
You are a Tax Advisor in India. Your task is to analyze my financial details and suggest which tax regime (Old Tax Regime 2023-24 or New Tax Regime 2024-25) allows me to save the maximum tax. Use the following financial details:

      Gross Salary: {"_id":"67c76453a9b0ac6e5e1e3b75","assessment_year":"N/A","gross_salary":"2557983.00","hra":"180150.00","travel_allowance":"0.00","gratuity":"0.00","leave_encashment":"0.00","standard_deduction":0,"professional_tax":"2400.00","other_income":"0.00","section_80C":"150000.00","section_80CCC":"0.00","section_80D":"0.00","section_80E":"0.00","section_80G":"0.00","section_80TTA":"0.00","rebate_87A":"0.00","additional_cess_info":"18605.00"}
      Total Investments: {"property":{"_id":"67e2d90a97006bf0bebcd6cf","rent_evolved":21000,"tax_paid":75000,"property_loan_interest":8.9,"__v":0},"agriculture":{"_id":"67f8038b4d12019a3197dc86","income_earned":45000,"expenses_incurred":5000,"__v":0},"capital-gains":{"_id":"67b387fd97e5559de3da9b38","shares":1200,"equity_mutual_funds":1233,"real_estate":166,"gold":12900,"listed_bonds":1233,"__v":0},"other":null}

      Breakdown of my current investments:
      - Property: {"_id":"67e2d90a97006bf0bebcd6cf","rent_evolved":21000,"tax_paid":75000,"property_loan_interest":8.9,"__v":0}
      - Agriculture: {"_id":"67f8038b4d12019a3197dc86","income_earned":45000,"expenses_incurred":5000,"__v":0}
      - Capital Gains: {"_id":"67b387fd97e5559de3da9b38","shares":1200,"equity_mutual_funds":1233,"real_estate":166,"gold":12900,"listed_bonds":1233,"__v":0}
      - Other: {}

      Based on this information, calculate the tax liability under:
      - **Old Tax Regime (2023-24)**: Include applicable deductions (e.g., Standard Deduction, Section 80C) and tax rates.
      - **New Tax Regime (2024-25)**: Apply the updated tax slabs with no deductions except where applicable.

     **IMPORTANT**  NOTE: Provide your response in the following JSON format only/-- (fill in all values as strings, use "N/A" if not applicable, and ensure calculations are accurate):
      {
        "Old_Tax_Regime_2023_24": {
          "Income_from_Property": {
            "Amount": "<amount>",
            "Tax_Rate": "<tax_rate>",
            "Tax_Amount": "<tax_amount>"
          },
          "Agricultural_Income": {
            "Amount": "<amount>",
            "Tax_Rate": "<tax_rate>",
            "Tax_Amount": "<tax_amount>"
          },
          "Capital_Gains": {
            "Breakdown": {
              "Shares": "<amount>",
              "Equity_Mutual_Funds": "<amount>",
              "Real_Estate": "<amount>",
              "Gold": "<amount>"
            },
            "Total": "<total_amount>",
            "Tax_Rate": "<tax_rate>",
            "Tax_Amount": "<tax_amount>"
          },
          "Other_Income": {
            "Amount": "<amount>",
            "Tax_Amount": "<tax_amount>"
          },
          "Deductions": {
            "Standard_Deduction": "<amount>",
            "Section_80C": "<amount>",
            "Total_Deductions": "<total_amount>"
          },
          "Net_Taxable_Income": "<net_taxable_income>",
          "Additional_Tax_Rate": "<additional_rate>",
          "Total_Tax": "<total_tax>"
        },
        "New_Tax_Regime_2024_25": {
          "Income_from_Property": {
            "Amount": "<amount>",
            "Tax_Rate": "<tax_rate>",
            "Tax_Amount": "<tax_amount>"
          },
          "Agricultural_Income": {
            "Amount": "<amount>",
            "Tax_Rate": "<tax_rate>",
            "Tax_Amount": "<tax_amount>"
          },
          "Capital_Gains": {
            "Breakdown": {
              "Shares": "<amount>",
              "Equity_Mutual_Funds": "<amount>",
              "Real_Estate": "<amount>",
              "Gold": "<amount>"
            },
            "Total": "<total_amount>",
            "Tax_Rate": "<tax_rate>",
            "Tax_Amount": "<tax_amount>"
          },
          "Other_Income": {
            "Amount": "<amount>",
            "Tax_Amount": "<tax_amount>"
          },
          "Deductions": {
            "Standard_Deduction": "<amount>",
            "Section_80C": "<amount>",
            "Total_Deductions": "<total_amount>"
          },
          "Net_Taxable_Income": "<net_taxable_income>",
          "Additional_Tax_Rate": "<additional_rate>",
          "Total_Tax": "<total_tax>"
        },
        "Comparison": {
          "Old_Regime_Total_Tax": "<old_total_tax>",
          "New_Regime_Total_Tax": "<new_total_tax>",
          "Tax_Saving": "<tax_saving_amount>",
          "Recommended_Regime": "<Old Tax Regime 2023-24 or New Tax Regime 2024-25>"
        }
      }

      Ensure the response is concise (under 2000 words), accurate ONLY IN JSON FORMAT GIVEN, Final amounts in both of the calculations should not be the same, includes all calculations. Use Indian tax rules and slabs for 2023-24 (Old) and 2024-25 (New).
"""

# Prepare the payload for the /generate endpoint
payload = {
    "prompt": prompt
}

# Send the POST request to the FastAPI backend
try:
    response = requests.post(FASTAPI_URL, json=payload)
    response.raise_for_status()  # Raise an exception for bad status codes (e.g., 4xx, 5xx)

    # Parse the response JSON
    result = response.json()
    tax_comparison = result["response"]

    print("Raw response from FastAPI:", tax_comparison)  # Debugging

    # Apply regex to extract JSON from the response
    json_pattern = r'\{(?:[^{}]|\{[^{}]*\})*\}'  # Same pattern as in /save-response
    match = re.search(json_pattern, tax_comparison, re.DOTALL)

    if match:
        json_string = match.group(0)  # Extract the matched JSON string
        print("Extracted JSON string:", json_string)  # Debugging
        try:
            # Validate and parse the extracted JSON
            tax_comparison_json = json.loads(json_string)
            # Pretty-print the response
            print("Generated Tax Comparison:")
            print(json.dumps(tax_comparison_json, indent=4))
        except json.JSONDecodeError as e:
            print(f"Error decoding extracted JSON: {e}")
            print("Raw extracted string:", json_string)
    else:
        print("No valid JSON found in the response")
        print("Raw response:", tax_comparison)

except requests.exceptions.RequestException as e:
    print(f"Error sending request to FastAPI: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding response JSON: {e}")
    print("Raw response:", tax_comparison)
except KeyError as e:
    print(f"Error: 'response' key not found in the response: {e}")
    print("Raw response:", result)

Raw response from FastAPI: To determine which tax regime allows you to save the maximum tax, I need to calculate the total tax liability under both the Old Tax Regime (2023-24) and the New Tax Regime (2024-25). Given your income and investments, we will proceed with the calculations.
```json
{
  "Old_Tax_Regime_2023_24": {
    "Income_from_Property": {
      "Amount": "75000",
      "Tax_Rate": "20%",
      "Tax_Amount": "15000"
    },
    "Agricultural_Income": {
      "Amount": "45000",
      "Tax_Rate": "20%",
      "Tax_Amount": "9000"
    },
    "Capital_Gains": {
      "Breakdown": {
        "Shares": "1200",
        "Equity_Mutual_Funds": "1233",
        "Real_Estate": "166",
        "Gold": "12900"
      },
      "Total": "14500",
      "Tax_Rate": "30%",
      "Tax_Amount": "4350"
    },
    "Other_Income": {
      "Amount": "0",
      "Tax_Amount": "0"
    },
    "Deductions": {
      "Standard_Deduction": "0",
      "Section_80C": "150000",
      "Total_Deductions": "150000"